In [1]:
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense
import numpy
import os
import videoto3d
import numpy as np

In [2]:
json_file = open('3dcnnmodel.json', 'r')
loaded_model_json = json_file.read()
loaded_model_json

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 32, 32, 10, 3], "dtype": "float32", "sparse": false, "ragged": false, "name": "conv3d_input"}}, {"class_name": "Conv3D", "config": {"name": "conv3d", "trainable": true, "batch_input_shape": [null, 32, 32, 10, 3], "dtype": "float32", "filters": 32, "kernel_size": [3, 3, 3], "strides": [1, 1, 1], "padding": "same", "data_format": "channels_last", "dilation_rate": [1, 1, 1], "groups": 1, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Activation", "config": {"name": "activation", "trainable": true, "dtype": "float32", "activation": "relu"}}, {"class_name": "Conv3D", 

In [3]:
json_file.close()

In [4]:
loaded_model = model_from_json(loaded_model_json)
loaded_model

In [5]:
loaded_model.load_weights("3dcnnmodel-gpu.hd5")

In [6]:
#check of weigths of biases of first layer
#weights, biases = loaded_model.layers[0].get_weights()

In [166]:
img_rows, img_cols, frames = 32, 32, 10
filepath= r'C:\Users\MY PC\3d-cnn-action-recognition\sphar\valid\stealing\uccrime_Robbery136_x264.mp4'
vid3d = videoto3d.Videoto3D(img_rows, img_cols, frames)
x = []
video_array = vid3d.video3d(filepath, color=True, skip=True)
x.append(video_array)
x_arr = np.array(x).transpose((0, 2, 3, 4, 1))
X = x_arr.reshape((x_arr.shape[0], img_rows, img_cols, frames, 3))
X = X.astype('float32')

In [167]:
classes = [
    "carcrash", "falling", "hitting", "igniting", "kicking", 
    "luggage", "murdering", "panicking", "running", "sitting", 
    "stealing", "vandalizing", "walking"
]

In [168]:
prediction = loaded_model.predict(X)

1/1 [==============================] - 0s 21ms/step


In [169]:
top_5_indices = np.argsort(prediction[0])[::-1][:5]
top_5_classes = [classes[i] for i in top_5_indices]
top_5_probabilities = [prediction[0][i] for i in top_5_indices]

In [170]:
print("Top 5 Predictions:")
for i, (class_name, probability) in enumerate(zip(top_5_classes, top_5_probabilities), 1):
    print(f"{i}. Class: {class_name}, Probability: {probability:.4f}")

Top 5 Predictions:
1. Class: stealing, Probability: 0.8209
2. Class: vandalizing, Probability: 0.0916
3. Class: igniting, Probability: 0.0547
4. Class: hitting, Probability: 0.0176
5. Class: carcrash, Probability: 0.0084


In [171]:
from IPython.display import HTML
from base64 import b64encode

# Function to display a frame from the video
def display_video_frame(filepath, frame_number):
    video_tag = '<video controls src="data:video/mp4;base64,{0}" type="video/mp4">'.format(
        b64encode(open(filepath, "rb").read()).decode('utf-8'))
    display(HTML(video_tag))

# Display frame number 100 from the video
display_video_frame(filepath, 10)